In [1]:
import pandas as pd
import csv as cv
import numpy as np
from ast import literal_eval
import re
from sklearn.tree import _tree
import time

In [111]:
def tree_to_code(tree, feature_names):
    f = open('TreeOutput.txt', 'w')
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    f.write("def tree({}):".format(", ".join(feature_names)))
    f.write("\n")

    def recurse(node, depth):
        indent = "  " * depth
        
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            f.write("{}if {} <= {}:".format(indent, name, threshold))
            f.write("\n")
            f.write("{}".format(indent)+"{")
            f.write("\n")
            
            recurse(tree_.children_left[node], depth + 1)
            f.write("{}".format(indent)+"}")
            f.write("\n")

            f.write("{}else:  # if {} > {}".format(indent, name, threshold))
            f.write("\n")

            f.write("{}".format(indent)+"{")
            f.write("\n")
            
            recurse(tree_.children_right[node], depth + 1)

            f.write("{}".format(indent)+"}")
            f.write("\n")
            
        else:
            f.write("{}return {}".format(indent, tree_.value[node][0]))
            f.write("\n")

    recurse(0, 1)
    f.close() 


def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1


def funcConvBranch(single_branch, dfT, rep, tree_no, num_class):
    with open('feNameType.csv') as csv_file:
        reader = cv.reader(csv_file)
        feName_type = dict(reader)
    with open('param_dict.csv') as csv_file:
        reader = cv.reader(csv_file)
        paramDict = dict(reader)

    f = open('DecSmt.smt2', 'a') 
    f.write("(assert (=> (and ")
    for i in range(0, len(single_branch)):
        temp_Str = single_branch[i]
        if 'if' in temp_Str:
            sign = re.search(r'>=|<=|>|<', temp_Str).group(0)    
            #fe_name =re.search(r'if(.*)[ sign]', temp_Str).group(1).split(sign)[0].strip()   
            fe_name = temp_Str.split('if')[1].split(sign)[0].strip()
            #print(fe_name)
            data_type = feName_type[fe_name]
            digit = float(re.search(r'[sign ][+-]?([0-9]*[.])?[0-9]+', temp_Str).group(0))
            #digit = format(digit, '.10f')
            #if 'int' in data_type:
            #    digit = int(round(digit))
            digit = str(digit)
            f.write("(" + sign + " "+ fe_name +str(rep)+" " + digit +") ") 

        elif 'return' in temp_Str:
            if paramDict['regression'] == 'yes':
                digit_class = float(re.search(r'[+-]?([0-9]*[.])?[0-9]+', temp_Str).group(0))
                f.write(") (= value"+str(rep)+str(tree_no)+" "+str(format(digit_class, '.7f'))+")))\n")
            else:    
                class_array = np.array(re.findall(r'\d+', temp_Str))
                class_array =[int(k) for k in class_array]
                f.write(') (and ')
                for i in range(0, num_class):
                    f.write('(= value'+str(rep)+str(tree_no)+str(i)+' '+str(class_array[i]/np.sum(class_array))+')')
                f.write(')))\n')
    f.close()


def funcGetBranch(sinBranch, dfT, rep, tree_no, num_class):
    sinBranch_set = set(sinBranch)
    return_pattern = re.compile(r'return (.*)')
    return_list = list(filter(return_pattern.match, sinBranch_set))
    if return_list:
        funcConvBranch(sinBranch, dfT, rep, tree_no, num_class)
            


def funcGenBranch(dfT, rep, paramDict, tree_no, num_class):
    with open('TreeOutput.txt') as f1:
        file_content = f1.readlines()
    file_content = [x.strip() for x in file_content]
    f1.close()
    noOfLines = file_len('TreeOutput.txt')
    temp_file_cont = ["" for x in range(noOfLines)]
    i = 1
    k = 0
    
    start_time = time.time()
    while i < noOfLines:
        j = k-1
        if temp_file_cont[j] == '}':
            funcGetBranch(temp_file_cont, dfT, rep, tree_no, num_class)
            while True:
                if temp_file_cont[j] == '{':
                    temp_file_cont[j] = ''
                    temp_file_cont[j-1] = ''
                    j = j-1
                    break  
                elif j>=0:
                    temp_file_cont[j] = ''
                    j = j-1
            k = j   
                
        else:    
            temp_file_cont[k] = file_content[i]
            k = k+1
            i = i+1
        #print('time after making a while true loop:', time.time()-start_time)    
    print('time in one loop:', time.time()-start_time)       
  
    if 'return' in file_content[1]:
        f = open('DecSmt.smt2', 'a')
        #f.write("(assert (= Class"+str(rep)+str(tree_no)+" "+str(digit)+"))")
        if paramDict['regression'] == 'yes':
            digit_class = float(re.search(r'[+-]?([0-9]*[.])?[0-9]+', file_content[1]).group(0))
            f.write(") (= value"+str(rep)+str(tree_no)+" "+str(format(digit_class, '.7f'))+")))\n")
        else:    
            class_array = np.array(re.findall(r'\d+', file_content[1]))
            class_array =[int(k) for k in class_array]
            f.write(') (and ')
            for i in range(0, num_class):
                f.write('(= value'+str(rep)+str(tree_no)+str(i)+' '+str(class_array[i]/np.sum(class_array))+')')
            f.write(')))\n')    
            f.close()
    else:    
        funcGetBranch(temp_file_cont, dfT, rep, tree_no, num_class)


def funcConv(dfT, no_of_instances, paramDict, tree_no, total_tree, num_class):
    
    with open('feNameType.csv') as csv_file:
        reader = cv.reader(csv_file)
        feName_type = dict(reader)
    #bound_cex = literal_eval(paramDict['bound_cex'])
    
    #f.write('(set-option :pp.decimal true) \n (set-option :pp.decimal_precision 8) \n')
    if tree_no == 0:
        f = open('DecSmt.smt2', 'w')
        for j in range(0, no_of_instances):
            for i in range (0, dfT.columns.values.shape[0]-1):
                tempStr = dfT.columns.values[i]
                fe_type = feName_type[tempStr]
                min_val = dfT.iloc[:, i].min()
                max_val = dfT.iloc[:, i].max() 
            
                if 'int' in fe_type:
                    f.write("(declare-fun " + tempStr+str(j)+ " () Int)")
                    f.write('\n')
                    #adding range
                    #if bound_cex:
                    #    f.write("(assert (and (>= "+tempStr+str(j)+" "+str(min_val)+")"+" "+"(<= "+tempStr+str(j)+" "+str(max_val)+")))")
                    #f.write('\n')
                elif('float' in fe_type):
                    f.write("(declare-fun " + tempStr+str(j)+ " () Real)")
                    f.write('\n')
                    #Adding range
                    #if bound_cex:
                        #f.write("(assert (and (>= "+tempStr+str(j)+" "+str(format(min_val, '.3f'))+")"+" "+"(<= "+tempStr+str(j)+" "+str(format(max_val, '.3f'))+")))")
                     #   f.write("(assert (and (>= " + tempStr + str(j) + " " + str(0.0) + ")" + " " + "(<= " + tempStr +
                      #          str(j) + " " + str(1.0) + ")))")
                    #f.write('\n') 
                
            f.write("; "+str(j)+"th element")
            f.write('\n')
        if paramDict['regression'] == 'no':
            for i in range(0, no_of_instances):
                for j in range(0, num_class):
                    for k in range(0, total_tree):
                        f.write("(declare-fun value" +str(i)+str(k)+str(j)+" () Real)\n")
                    f.write("(declare-fun temp_value" +str(i)+str(j)+" () Real)\n")    
        else:
            for i in range(0, no_of_instances):
                for j in range(0, total_tree):
                    f.write("(declare-fun value" +str(i)+str(j)+" () Real)\n")
            
        if 'int' in feName_type['Class']:
            f.write("(declare-fun Class"+str(i)+" () Int)\n")
        else:
            f.write("(declare-fun Class" +str(i)+" () Real)\n")
        f.write('\n')
           
        #f = open('DecSmt.smt2', 'a')
        #Writing the functions for computing absolute integer & real value
        f.write('(define-fun absoluteInt ((x Int)) Int \n')
        f.write('  (ite (>= x 0) x ''(- x))) \n')
        f.write('(define-fun absoluteReal ((x Real)) Real \n')
        f.write('  (ite (>= x 0) x (- x))) \n')
        f.write('(declare-fun constnt () Real)\n')
        f.write('(declare-fun temp () Real)\n')
        f.write(';--------'+str(tree_no)+'th tree encoding-----------\n')
        f.close()

    #Calling function to get the branch and convert it to z3 form,  creating alias
    if tree_no > 0:
        f = open('DecSmt.smt2', 'a')
        f.write(';--------'+str(tree_no)+'th tree encoding-----------\n')
        f.close()
    
    for i in range(0, no_of_instances):  
        if paramDict['regression'] == 'no':
            funcGenBranch(dfT, i, paramDict, tree_no, num_class)
        else:    
            funcGenBranch(dfT, i, paramDict, tree_no, 1)


def functree2LogicMain(tree, no_of_instances, tree_no, total_tree, df, num_class):
    with open('param_dict.csv') as csv_file:
        reader = cv.reader(csv_file)
        paramDict = dict(reader)    
    tree_to_code(tree, df.columns)
    funcConv(df, no_of_instances, paramDict, tree_no, total_tree, num_class)
    

In [112]:
from joblib import load
rf = load('rf_regression.joblib')
df_data = pd.read_csv('TestOracleRegression.csv')

In [113]:
start_timey= time.time()
num_class = df_data['Class'].nunique()
for i in range(0, len(rf.estimators_)):
    functree2LogicMain(rf.estimators_[i], 1, i, len(rf.estimators_), df_data, num_class)
print('time in total', time.time()-start_timey)    

time in one loop: 0.0
time in one loop: 0.0
time in one loop: 0.0
time in total 0.046863555908203125


In [106]:
#RF classification
f1 = open('DecSmt.smt2', 'a')
f1.write('\n\n')
for i in range(0, 1):
    for j in range(0, df_data['Class'].nunique()):
        f1.write('(assert (= temp_value'+str(i)+str(j)+' (/ (+ ')
        for k in range(0, len(rf.estimators_)):
            f1.write('value'+str(i)+str(k)+str(j)+' ')
        f1.write(') '+str(len(rf.estimators_))+')))\n')
        
    f1.write('\n\n(assert (or ')
    for j in range(0, df_data['Class'].nunique()):
        f1.write('\n(and (= Class'+str(i)+' '+str(j)+') (and ')
        for k in range(0, df_data['Class'].nunique()):
            if j != k:
                f1.write('(> temp_value'+str(i)+str(j)+' ')
                f1.write('temp_value'+str(i)+str(k)+')')
        f1.write('))')
    f1.write('\n(and (= Class'+str(i)+' 0'+') (= ')
    for k in range(0, df_data['Class'].nunique()):
        f1.write(' temp_value'+str(i)+str(k))
    f1.write('))')    
    f1.write('))\n')        
    f1.write(';Encode property here\n')
    #f1.write('(assert (= Class'+str(i)+' (to_int (/ temp_class'+str(i)+' '+str(len(rf.estimators_))+')))) \n')

f1.write('(check-sat)\n')
f1.write('(get-model)')
f1.close()    
            

In [114]:
#RF_regression
f1 = open('DecSmt.smt2', 'a')
f1.write('\n\n')
for i in range(0, 1):
    f1.write('(assert (= Class'+str(i)+' (/ (+ ')
    for j in range(len(rf.estimators_)):
        f1.write('value'+str(i)+str(j)+' ')
    f1.write(') '+str(len(rf.estimators_))+')))\n')
f1.write(';Encode property here\n')

f1.write('(check-sat)\n')
f1.write('(get-model)')
f1.close()    
            